# Урок 1. Основы обучения нейронных сетей

## 1.1. Нейроны

<img src=pics/01.png>

1. Значения входов умножаются на веса:

    $x_1 \rightarrow x_1 \times w_1$, $x_2 \rightarrow x_2 \times w_2$

2. Взвешенные входы складываются, и к ним прибавляется значение порога b:

    $x_1 \times w_1 + x_2 \times w_2 + b$

3. Полученная сумма проходит через функцию активации:
   
    $y=f(x_1 \times w_1 + x_2 \times w_2 + b)$

### Пример математический

- Веса (в векторном виде): $w = [0,1]$
- Порог: $b = 4$
- Входные данные: $x=[2,3]$

  Скалярное произведение векторов:
  $$(w \cdot x) + b = ((w_1 \times x_1) + (w_2 \times x_2)) + b = 0 \times 2 + 1 \times 3 + 4 = 7$$

  Сигмоидная функция:
  $$y=f(w\cdot x+b)=f(7)\approx  0.999 $$

### Пример на пайтоне

In [ ]:
import numpy as np

In [2]:
def sigmoid(x):
  # Наша функция активации: f(x) = 1 / (1 + e^(-x))
  return 1 / (1 + np.exp(-x))

In [3]:
class Neuron:
  def __init__(self, weights, bias):
    self.weights = weights
    self.bias = bias

  def feedforward(self, inputs):
    # Умножаем входы на веса, прибавляем порог, затем используем функцию активации
    total = np.dot(self.weights, inputs) + self.bias
    return sigmoid(total)

weights = np.array([0, 1]) # w1 = 0, w2 = 1
bias = 4                   # b = 4
n = Neuron(weights, bias)

x = np.array([2, 3])       # x1 = 2, x2 = 3
print(n.feedforward(x))    # 0.9990889488055994

0.9990889488055994


## 1.2. ИНС

### Пример математический

<img src=pics/02.png>

- Веса : $w = [0,1]$
- Порог: $b = 0$
- Входные данные: $x=[2,3]$
- Выходы первого слоя: $h_1$, $h_2$
- Выход последнего слоя: $o_1$

  Скалярное произведение векторов через сигмоидную функцию:
  $$h_1=h_2=f(w \cdot x + b) = f((0 \times 2) + (1*3) + 0) = f(3) \approx 0.9526$$
  $$o_1= f(w \cdot [h_1,h_2] + b) = f((0\times h_1) + (1\times h_2)+0)=f(0.9526)\approx 0.7216$$

### Код на пайтоне

In [1]:
import numpy as np

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

class Neuron:
  def __init__(self, weights, bias):
    self.weights = weights
    self.bias = bias

  def feedforward(self, inputs):
    total = np.dot(self.weights, inputs) + self.bias
    return sigmoid(total)

class OurNeuralNetwork:
  '''
  Нейронная сеть с:
    - 2 входами
    - скрытым слоем с 2 нейронами (h1, h2)
    - выходным слоем с 1 нейроном (o1)
  Все нейроны имеют одинаковые веса и пороги:
    - w = [0, 1]
    - b = 0
  '''
  def __init__(self):
    weights = np.array([0, 1])
    bias = 0

    # Используем класс Neuron из предыдущего раздела
    self.h1 = Neuron(weights, bias)
    self.h2 = Neuron(weights, bias)
    self.o1 = Neuron(weights, bias)

  def feedforward(self, x):
    out_h1 = self.h1.feedforward(x)
    out_h2 = self.h2.feedforward(x)

    # Входы для o1 - это выходы h1 и h2
    out_o1 = self.o1.feedforward(np.array([out_h1, out_h2]))

    return out_o1

network = OurNeuralNetwork()
x = np.array([2, 3])
print(network.feedforward(x)) # 0.7216325609518421

0.7216325609518421


## 1.3 Потери ИНС

Допустим, у нас есть следующие измерения:
<table>
              <tr>
                      <td><b>Имя</b></td>
                      <td><b>Вес (в фунтах)</b></td>
                      <td><b>Рост (в дюймах)</b></td>
                      <td><b>Пол</b></td>
                  </tr>
              <tr>
                      <td>Алиса</td>
                      <td>133 (54.4 кг)</td>
                      <td>65 (165,1 см)</td>
                      <td>Ж</td>
                  </tr>
              <tr>
                      <td>Боб</td>
                      <td>160 (65,44 кг)</td>
                      <td>72 (183 см)</td>
                      <td>М</td>
                  </tr>
              <tr>
                      <td>Чарли</td>
                      <td>152 (62.2 кг)</td>
                      <td>70 (178 см)</td>
                      <td>М</td>
                  </tr>
              <tr>
                      <td>Диана</td>
                      <td>120 (49 кг)</td>
                      <td>60 (152 см)</td>
                      <td>Ж</td>
                  </tr>
</table>

Обучим  ИНС предсказывать пол человека по росту и весу

<img src=pics/03.png>

Представим мужской пол как 0, женский – как 1, а также сдвинем данные, чтобы их было проще использовать:

<table>
                  <tr>
                          <td><b>Имя</b></td>
                          <td><b>Вес (минус 135)</b></td>
                          <td><b>Рост (минус 66)</b></td>
                          <td><b>Пол</b></td>
                      </tr>
                  <tr>
                          <td>Алиса</td>
                          <td>-2</td>
                          <td>-1</td>
                          <td>1</td>
                      </tr>
                  <tr>
                          <td>Боб</td>
                          <td>25</td>
                          <td>6</td>
                          <td>0</td>
                      </tr>
                  <tr>
                          <td>Чарли</td>
                          <td>17</td>
                          <td>4</td>
                          <td>0</td>
                      </tr>
                  <tr>
                          <td>Диана</td>
                          <td>-15</td>
                          <td>-6</td>
                          <td>1</td>
                      </tr>   
</table>

Для расчета потерь используют среднюю квадратичную ошибку (mean squared error, MSE):

$$MSE=\frac{1}{n}\sum_{i=1}^{n}(y_true-y_pred)^2$$

- n – количество измерений, в нашем случае 4 (Алиса, Боб, Чарли и Диана).
- y представляет предсказываемое значение, Пол.
- $y_true$ – истинное значение переменной ("правильный ответ"). Например, для Алисы ytrue будет равна 1 (женский пол).
- $y_pred$ – предсказанное значение переменной. Это то, что выдаст наша нейронная сеть.
- $(y_true-y_pred)^2 - квадратичная ошибка

### Пример расчета потерь

Предположим, ИНС всегда выдает 0 – иными словами, она уверена, что все люди мужчины.
    
<table>
                  <tr>
                          <td><b>Имя</b></td>
                          <td><b>Ytrue</b></td>
                          <td><b>Ypred</b></td>
                          <td><b>(Ytrue-Ypred)^2</b></td>
                      </tr>
                  <tr>
                          <td>Алиса</td>
                          <td>1</td>
                          <td>0</td>
                          <td>1</td>
                      </tr>
                  <tr>
                          <td>Боб</td>
                          <td>0</td>
                          <td>0</td>
                          <td>0</td>
                      </tr>
                  <tr>
                          <td>Чарли</td>
                          <td>0</td>
                          <td>0</td>
                          <td>0</td>
                      </tr>
                  <tr>
                          <td>Диана</td>
                          <td>1</td>
                          <td>0</td>
                          <td>1</td>
                      </tr>
    </table>

$$MSE=\frac{1}{4}(1+0+0+1)=0.5$$

### Функция потерь на пайтоне

In [4]:
import numpy as np

def mse_loss(y_true, y_pred):
  # y_true и y_pred - массивы numpy одинаковой длины.
  return ((y_true - y_pred) ** 2).mean()

y_true = np.array([1, 0, 0, 1])
y_pred = np.array([0, 0, 0, 0])

print(mse_loss(y_true, y_pred)) # 0.5

0.5


### 1.4 Обучение ИНС

Если рассматривать, только Алису, то и квадратичная ошибка будет только для Алисы

Но, есть более эффективной метод - функция потерь, как функция весов и порогов.

Отметитим все веса и пороги ИНС:

<img src=pics/04.png>

Запишем функцию потерь как функцию от нескольких переменных:

$$L(w_1,w_2,w_3,w_4,w_5,w_6,b_1,b_2,b_3)$$

Предположим, хоим отрегулировать $w_1$. Как это повлияет на значение потери L?


### Код ИНС

In [6]:
import numpy as np

def sigmoid(x):
  # Сигмоидная функция активации: f(x) = 1 / (1 + e^(-x))
  return 1 / (1 + np.exp(-x))

def deriv_sigmoid(x):
  # Производная сигмоиды: f'(x) = f(x) * (1 - f(x))
  fx = sigmoid(x)
  return fx * (1 - fx)

def mse_loss(y_true, y_pred):
  # y_true и y_pred - массивы numpy одинаковой длины.
  return ((y_true - y_pred) ** 2).mean()

class OurNeuralNetwork:
  '''
  Нейронная сеть с:
    - 2 входами
    - скрытым слоем с 2 нейронами (h1, h2)
    - выходной слой с 1 нейроном (o1)

  *** DISCLAIMER ***:
  Следующий код простой и обучающий, но НЕ оптимальный.
  Код реальных нейронных сетей совсем на него не похож. НЕ копируйте его! 
  Изучайте и запускайте его, чтобы понять, как работает эта нейронная сеть.
  '''
  def __init__(self):
    # Веса
    self.w1 = np.random.normal()
    self.w2 = np.random.normal()
    self.w3 = np.random.normal()
    self.w4 = np.random.normal()
    self.w5 = np.random.normal()
    self.w6 = np.random.normal()

    # Пороги
    self.b1 = np.random.normal()
    self.b2 = np.random.normal()
    self.b3 = np.random.normal()

  def feedforward(self, x):
    # x is a numpy array with 2 elements.
    h1 = sigmoid(self.w1 * x[0] + self.w2 * x[1] + self.b1)
    h2 = sigmoid(self.w3 * x[0] + self.w4 * x[1] + self.b2)
    o1 = sigmoid(self.w5 * h1 + self.w6 * h2 + self.b3)
    return o1

  def train(self, data, all_y_trues):
    '''
    - data - массив numpy (n x 2) numpy, n = к-во наблюдений в наборе. 
    - all_y_trues - массив numpy с n элементами.
      Элементы all_y_trues соответствуют наблюдениям в data.
    '''
    learn_rate = 0.1
    epochs = 1000 # сколько раз пройти по всему набору данных 

    for epoch in range(epochs):
      for x, y_true in zip(data, all_y_trues):
        # --- Прямой проход (эти значения нам понадобятся позже)
        sum_h1 = self.w1 * x[0] + self.w2 * x[1] + self.b1
        h1 = sigmoid(sum_h1)

        sum_h2 = self.w3 * x[0] + self.w4 * x[1] + self.b2
        h2 = sigmoid(sum_h2)

        sum_o1 = self.w5 * h1 + self.w6 * h2 + self.b3
        o1 = sigmoid(sum_o1)
        y_pred = o1

        # --- Считаем частные производные.
        # --- Имена: d_L_d_w1 = "частная производная L по w1"
        d_L_d_ypred = -2 * (y_true - y_pred)

        # Нейрон o1
        d_ypred_d_w5 = h1 * deriv_sigmoid(sum_o1)
        d_ypred_d_w6 = h2 * deriv_sigmoid(sum_o1)
        d_ypred_d_b3 = deriv_sigmoid(sum_o1)

        d_ypred_d_h1 = self.w5 * deriv_sigmoid(sum_o1)
        d_ypred_d_h2 = self.w6 * deriv_sigmoid(sum_o1)

        # Нейрон h1
        d_h1_d_w1 = x[0] * deriv_sigmoid(sum_h1)
        d_h1_d_w2 = x[1] * deriv_sigmoid(sum_h1)
        d_h1_d_b1 = deriv_sigmoid(sum_h1)

        # Нейрон h2
        d_h2_d_w3 = x[0] * deriv_sigmoid(sum_h2)
        d_h2_d_w4 = x[1] * deriv_sigmoid(sum_h2)
        d_h2_d_b2 = deriv_sigmoid(sum_h2)

        # --- Обновляем веса и пороги
        # Нейрон h1
        self.w1 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w1
        self.w2 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w2
        self.b1 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_b1

        # Нейрон h2
        self.w3 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w3
        self.w4 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w4
        self.b2 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_b2

        # Нейрон o1
        self.w5 -= learn_rate * d_L_d_ypred * d_ypred_d_w5
        self.w6 -= learn_rate * d_L_d_ypred * d_ypred_d_w6
        self.b3 -= learn_rate * d_L_d_ypred * d_ypred_d_b3

      # --- Считаем полные потери в конце каждой эпохи
      if epoch % 10 == 0:
        y_preds = np.apply_along_axis(self.feedforward, 1, data)
        loss = mse_loss(all_y_trues, y_preds)
        print("Epoch %d loss: %.3f" % (epoch, loss))

# Определим набор данных
data = np.array([
  [-2, -1],  # Алиса
  [25, 6],   # Боб
  [17, 4],   # Чарли
  [-15, -6], # Диана
])
all_y_trues = np.array([
  1, # Алиса
  0, # Боб
  0, # Чарли
  1, # Диана
])

# Обучаем нашу нейронную сеть!
network = OurNeuralNetwork()
network.train(data, all_y_trues)

Epoch 0 loss: 0.188
Epoch 10 loss: 0.147
Epoch 20 loss: 0.116
Epoch 30 loss: 0.093
Epoch 40 loss: 0.076
Epoch 50 loss: 0.063
Epoch 60 loss: 0.054
Epoch 70 loss: 0.046
Epoch 80 loss: 0.040
Epoch 90 loss: 0.036
Epoch 100 loss: 0.032
Epoch 110 loss: 0.029
Epoch 120 loss: 0.026
Epoch 130 loss: 0.024
Epoch 140 loss: 0.022
Epoch 150 loss: 0.020
Epoch 160 loss: 0.019
Epoch 170 loss: 0.017
Epoch 180 loss: 0.016
Epoch 190 loss: 0.015
Epoch 200 loss: 0.014
Epoch 210 loss: 0.014
Epoch 220 loss: 0.013
Epoch 230 loss: 0.012
Epoch 240 loss: 0.012
Epoch 250 loss: 0.011
Epoch 260 loss: 0.011
Epoch 270 loss: 0.010
Epoch 280 loss: 0.010
Epoch 290 loss: 0.009
Epoch 300 loss: 0.009
Epoch 310 loss: 0.009
Epoch 320 loss: 0.008
Epoch 330 loss: 0.008
Epoch 340 loss: 0.008
Epoch 350 loss: 0.008
Epoch 360 loss: 0.007
Epoch 370 loss: 0.007
Epoch 380 loss: 0.007
Epoch 390 loss: 0.007
Epoch 400 loss: 0.006
Epoch 410 loss: 0.006
Epoch 420 loss: 0.006
Epoch 430 loss: 0.006
Epoch 440 loss: 0.006
Epoch 450 loss: 0.006

<img src=pics/05.png>

### Предсказание пола

In [7]:
# Делаем пару предсказаний
emily = np.array([-7, -3]) # 128 фунтов (52.35 кг), 63 дюйма (160 см)
frank = np.array([20, 2])  # 155 pounds (63.4 кг), 68 inches (173 см)
print("Эмили: %.3f" % network.feedforward(emily)) # 0.951 - Ж
print("Фрэнк: %.3f" % network.feedforward(frank)) # 0.039 - М

Эмили: 0.947
Фрэнк: 0.039


# Домашняя работа

ДЗ от меня - доделать НС с урока, добавить метод predict и проверить на своих собственных данных, оценить accuracy.

Как вариант, опционально кто сможет для решения этих входных данных обучите НС mlpclassifier от sklearn.

Сдача только через гитхаб либо ссылкой на блокнот Google Colab.

In [38]:
def predict(weight, height):
    # Переводим рост из сантиметров в футы
    height_ft = height / 2.54
    # Переводим вес из килограммов в фунты
    weight_lb = weight * 2.448
    #return weight_lb, height_ft
    who = np.array([weight_lb-135, height_ft-66])
    if network.feedforward(who) >= 0.5:
        return 'Women'
    else: 
        return 'Man'

predict(77, 185)

'Women'